In [1]:
import pyreadr
import pandas as pd
import numpy as np
import math

In [2]:
data_path = "/data/IDEA_DeFi_Research/LTM/Data/Lending_Protocols/Aave/V2/Mainnet"

include_user_features = True
include_time_features = True
include_market_features = True
include_exo_features = True

feature_extension = ""
if include_user_features:
    feature_extension += "_user"
if include_market_features:
    feature_extension += "_market"
if include_time_features:
    feature_extension += "_time"
if include_exo_features:
    feature_extension += "_exoLagged"



file_path = f"{data_path}/transactions_user_market_time_exoLagged.rds"
train_path = f"{data_path}/transactions{feature_extension}_train.csv"
test_path = f"{data_path}/transactions{feature_extension}_test.csv"
seq_len=10
train_test_thres_year=2024

In [3]:
fullData = pyreadr.read_r(file_path)[None]  

In [4]:
# Drop the unnecessary columns:
# First, we can definitely drop any columns for which ALL values are NA:
all_na_columns = fullData.columns[fullData.isna().all()].tolist()

# We build the list of user-level features here, in case we want to drop them easily for some experiments:
user_features = [col for col in fullData.columns if col.startswith('user') and col != "user"]

# We build the list of time features in case we want to drop them easily for some experiments:
time_features = ["timeOfDay", "dayOfWeek", "dayOfMonth", 
    "dayOfYear", "quarter", "dayOfQuarter",
    "sinTimeOfDay", "cosTimeOfDay", "sinDayOfWeek",
    "cosDayOfWeek", "sinDayOfMonth", "cosDayOfMonth",
    "sinDayOfQuarter", "cosDayOfQuarter", "sinDayOfYear",
    "cosDayOfYear", "sinQuarter", "cosQuarter", "isWeekend"]

# We are going to drop the market features for now, because they are similar to exogenous
# features and we will likely handle them differently later on.
market_features = [col for col in fullData.columns if col.startswith('market')]

# We are going to drop the exogenous features for now and handle them separately once we have 
# the rest of the model working.
exo_features = [col for col in fullData.columns if col.startswith('exo')]

# There might be additional, hand-selected columns we want to drop just because they are not useful or because
# they are going to be re-created in a more general way:
other_columns_to_drop = ['logAmountUSD', 'logAmount', 'logAmountETH']

# Concatenate these lists and then drop the columns:
columns_to_drop = all_na_columns + other_columns_to_drop
if not include_user_features:
    columns_to_drop += user_features
if not include_market_features:
    columns_to_drop += market_features
if not include_time_features:
    columns_to_drop += time_features
if not include_exo_features:
    columns_to_drop += exo_features

data = fullData.drop(columns=columns_to_drop, errors="ignore")

In [5]:
data['timeFeature'] = ((data['timestamp'] - data['timestamp'].min())//60).astype(int)
data['Year'] = pd.to_datetime(data['timestamp'], unit='s').dt.year

In [6]:
# Let's make sure certain columns are cast properly to categorical columns so they 
# are handled appropriately when building the vocabulary down the line.
categorical_columns = ['fromState', 'toState', 'borrowRateMode', 'reserve', 'type', 
                       'collateralReserve', 'borrowRateModeTo', 'borrowRateModeFrom',
                       'coinType', 
                       "dayOfWeek", "dayOfMonth", "dayOfYear", "quarter", "dayOfQuarter", 'isWeekend', 'Year',
                       'userReserveMode', 'userCoinTypeMode', 'userIsNew']

for col in categorical_columns:
    data[col] = data[col].astype('category')


In [7]:
sort_columns =  ['user', 'timeFeature']
new_data = data.sort_values(by=sort_columns)

In [ ]:
# columns we specifically don't want to log-transform:
columns_not_to_log = ['timeFeature']
"""
# List of keywords indicating a column should be log-transformed
keywords_to_log = ['amount', 'sum', 'rate', 'count', 'activedays', 'sin', 'cos']

# Identify columns to log-transform
columns_to_log = [col for col in new_data.columns if any(keyword in col.lower() for keyword in keywords_to_log)]
columns_to_log = list(set(columns_to_log))  # Remove duplicates
columns_to_log = [x for x in columns_to_log if isinstance(x, (int, float))]
"""

# We want to log-transform all numeric columns:
columns_to_log = new_data.select_dtypes(include=['number']).columns.tolist()

# Apply log transformation to identified columns
for col in columns_to_log:
    if col in columns_not_to_log:
        continue
    new_data[col] = new_data[col].apply(lambda x: np.log(x) if x > 0 else np.nan)  # Use 1 for non-positive values
    
new_data['rowNumber'] = np.arange(len(new_data))



In [ ]:
# List of columns to move to the front
first_cols = ['rowNumber', 'user', 'timestamp', 'id']
# Rearrange the columns
rearranged_columns = first_cols + [col for col in new_data.columns if col not in first_cols]
new_data = new_data[rearranged_columns]

In [ ]:
train_data = new_data.loc[new_data['Year'].astype(int) < train_test_thres_year]
basic_test_data = new_data.loc[new_data['Year'].astype(int) >= train_test_thres_year]


In [ ]:
train_user = set(train_data['user'].unique())
test_user = set(basic_test_data['user'].unique())
train_test_user = train_user.intersection(test_user)
test_only_user = test_user.difference(train_user)
groupby_columns = ['user']

In [ ]:
def get_index(x, seq_len):
    return x.index[-(seq_len-1):]

In [ ]:
test_extra_index = train_data.loc[train_data['user'].isin(train_test_user)].groupby(groupby_columns).apply(get_index, seq_len)
test_extra_index = test_extra_index.explode()

In [ ]:
test_data = pd.concat([new_data.loc[test_extra_index], basic_test_data])

In [ ]:
test_data.sort_values(by=sort_columns, inplace=True)

In [ ]:
train_data.to_csv(train_path, index=False)
test_data.to_csv(test_path, index=False)